# Profiling Emax for Ambiguity

In [ ]:
import pandas as pd
import numpy as np
from robupy.get_worst_case import get_worst_case_probs

%load_ext nb_black
%load_ext snakeviz

In [2]:
draws = 500

<IPython.core.display.Javascript object>

In [3]:
v = np.arange(draws)
q = np.full(draws, 1 / draws)
eta = 0.1

<IPython.core.display.Javascript object>

In [4]:
# To use snakevize for get_worst_case_probs it is necessary to uncomment all @njit commands
# in the submodules/robupy directory
# %snakeviz get_worst_case_probs(v, q, eta)

<IPython.core.display.Javascript object>

In [5]:
# from respy.solve import calculate_expected_value_functions

<IPython.core.display.Javascript object>

In [6]:
# params, options, _ = rp.get_example_model("kw_94_two")
# options["simulation_agents"] = NUM_AGENTS
# options["n_periods"] = NUM_PERIODS

<IPython.core.display.Javascript object>

In [7]:
def test_sum(x, y):
    return x + y

<IPython.core.display.Javascript object>

In [8]:
def test_1(x, y):
    for i in range(1000):
        test_sum(x, y)
        np.arange(10000)
    x + y
    return x

<IPython.core.display.Javascript object>

In [9]:
# %snakeviz -t test_1(2,3)

<IPython.core.display.Javascript object>

In [ ]:
simulate = rp.get_simulate_func(params, options)

In [ ]:
def calculate_expected_value_functions(
    wages, nonpecs, continuation_values, draws, delta, eta, expected_value_functions
):
    r"""Calculate the expected maximum of value functions for a set of unobservables.

    The function takes an agent and calculates the utility for each of the choices, the
    ex-post rewards, with multiple draws from the distribution of unobservables and adds
    the discounted expected maximum utility under a worst case scenario of subsequent periods
    resulting from choices. Averaging over all maximum utilities yields the expected maximum
    utility of this state

    The underlying process in this function is called `Monte Carlo integration`_. The
    goal is to approximate an integral by evaluating the integrand at randomly chosen
    points. In this setting, one wants to approximate the expected maximum utility of
    the current state.

    Note that `wages` have the same length as `nonpecs` despite that wages are only
    available in some choices. Missing choices are filled with ones. In the case of a
    choice with wage and without wage, flow utilities are

    .. math::

        \text{Flow Utility} = \text{Wage} * \epsilon + \text{Non-pecuniary}
        \text{Flow Utility} = 1 * \epsilon + \text{Non-pecuniary}

    Parameters
    ----------
    wages : numpy.ndarray
        Array with shape (n_choices,) containing wages.
    nonpecs : numpy.ndarray
        Array with shape (n_choices,) containing non-pecuniary rewards.
    continuation_values : numpy.ndarray
        Array with shape (n_choices,) containing expected maximum utility for each
        choice in the subsequent period.
    draws : numpy.ndarray
        Array with shape (n_draws, n_choices).
    delta : float
        The discount factor.
    eta: float
        The size of the ambiguity set.

    Returns
    -------
    expected_value_functions : float
        Expected maximum utility of an agent.

    .. _Monte Carlo integration:
        https://en.wikipedia.org/wiki/Monte_Carlo_integration

    """
    n_draws, n_choices = draws.shape
    v = np.repeat(np.nan, n_draws)

    expected_value_functions[0] = 0

    for i in range(n_draws):

        max_value_functions = 0

        for j in range(n_choices):
            value_function, _ = aggregate_keane_wolpin_utility(
                wages[j], nonpecs[j], continuation_values[j], draws[i, j], delta
            )

            if value_function > max_value_functions:
                max_value_functions = value_function

        v[i] = max_value_functions

    if eta > 0:
        q = np.repeat(1.0 / n_draws, n_draws)
        p = get_worst_case_probs(v, q, eta)
        emax = np.dot(v, p)
    else:
        emax = np.mean(v)

    expected_value_functions[0] = emax